In [6]:
import sys

sys.path.append("..")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from openpoints.utils import EasyConfig, set_random_seed, dist_utils
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg
from openpoints.dataset import build_dataset_from_cfg

import torch
from torch_geometric.data import Data, Batch
import os
import glob

In [7]:
from torch_scatter import scatter
from openpoints.utils import (
    set_random_seed,
    save_checkpoint,
    load_checkpoint,
    resume_checkpoint,
    setup_logger_dist,
    cal_model_parm_nums,
    Wandb,
    generate_exp_directory,
    resume_exp_directory,
    EasyConfig,
    dist_utils,
    find_free_port,
    load_checkpoint_inv,
)
from openpoints.utils import AverageMeter, ConfusionMatrix, get_mious
from openpoints.dataset import build_dataloader_from_cfg, get_features_by_keys, get_class_weights
from openpoints.dataset.data_util import voxelize
from openpoints.dataset.semantic_kitti.semantickitti import (
    load_label_kitti,
    load_pc_kitti,
    remap_lut_read,
    remap_lut_write,
    get_semantickitti_file_list,
)
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg

In [8]:
cfg = EasyConfig()
cfg.load("../cfgs/nibio_mls/pointnext-xl.yaml", recursive=True)
cfg.dataset.common.data_root = os.path.join("../", cfg.dataset.common.data_root)

In [9]:
dataset = build_dataset_from_cfg(cfg.dataset.common, cfg.dataset.train)

../data/NIBIO_MLS/processed/nibio_mls_train_0.040_6m.pkl load successfully


In [10]:
dataset[0]

{'pos': array([[2.9629974, 1.421    , 4.719    ],
        [2.959999 , 1.3860002, 4.792    ],
        [2.9629974, 1.3400002, 4.672    ],
        ...,
        [1.2990036, 1.4440002, 3.8199997],
        [1.2850037, 1.3910003, 4.5439997],
        [1.3040009, 1.4580002, 3.725    ]], shape=(3139, 3), dtype=float32),
 'y': array([2, 2, 2, ..., 2, 2, 2], shape=(3139,)),
 'heights': tensor([[4.7190],
         [4.7920],
         [4.6720],
         ...,
         [3.8200],
         [4.5440],
         [3.7250]]),
 'x': array([[2.9629974, 1.421    , 4.719    ],
        [2.959999 , 1.3860002, 4.792    ],
        [2.9629974, 1.3400002, 4.672    ],
        ...,
        [1.2990036, 1.4440002, 3.8199997],
        [1.2850037, 1.3910003, 4.5439997],
        [1.3040009, 1.4580002, 3.725    ]], shape=(3139, 3), dtype=float32)}

In [11]:
model = build_model_from_cfg(cfg.model)
model = model.to(torch.device("cuda:0"))

In [ ]:
load_checkpoint(model, "../pointnext-xl_ckpt_best.pth")

In [ ]:
model.state_dict()

In [ ]:
num_points = 16000

pos = torch.randn(num_points, 3)

x = None

data = Data(pos=pos)

batch = Batch.from_data_list([data, data])
batch = batch.to(torch.device("cuda:0"))

In [ ]:
keys = dataset[0].keys() if callable(dataset[0].keys) else data.keys

In [ ]:
keys

In [12]:
cfg.distributed = False
val_loader = build_dataloader_from_cfg(
    1,
    cfg.dataset,
    cfg.dataloader,
    datatransforms_cfg=cfg.datatransforms,
    split="train",
    distributed=cfg.distributed,
)

../data/NIBIO_MLS/processed/nibio_mls_train_0.040_6m.pkl load successfully


In [13]:
test = next(iter(val_loader))
test

{'pos': array([[3.        , 2.7060003 , 2.4720001 ],
       [3.        , 0.928     , 0.89299965],
       [2.974     , 0.97800016, 0.32099962],
       ...,
       [1.293     , 4.3310003 , 2.5760002 ],
       [1.306     , 4.335     , 1.73      ],
       [1.304     , 4.337     , 1.685     ]],
      shape=(12766, 3), dtype=float32), 'y': array([4, 2, 2, ..., 2, 2, 2], shape=(12766,))}{'pos': array([[6.        , 2.2339993 , 0.5419998 ],
       [5.976     , 1.098999  , 0.18400002],
       [5.972     , 1.1309996 , 0.02499962],
       ...,
       [1.284     , 1.3759995 , 1.8069992 ],
       [1.311     , 1.3329992 , 1.868     ],
       [1.31      , 1.9539995 , 0.01799965]],
      shape=(4516, 3), dtype=float32), 'y': array([2, 2, 2, ..., 2, 2, 2], shape=(4516,))}
{'pos': array([[5.972     , 4.6720004 , 2.6599998 ],
       [5.987     , 4.3750005 , 5.4839993 ],
       [5.975     , 5.967001  , 1.0869999 ],
       ...,
       [1.281     , 3.4570003 , 0.31599998],
       [1.302     , 4.0850005 , 2.6

{'pos': tensor([[[-0.5166,  1.4814,  2.5766],
          [ 1.1278,  1.7540,  0.9303],
          [ 1.0922,  1.7178,  0.3414],
          ...,
          [-1.7778, -0.3406,  2.6815],
          [-1.7814, -0.3256,  1.8128],
          [-1.7815, -0.3236,  1.7574]]]),
 'y': tensor([[4, 2, 2,  ..., 2, 2, 2]]),
 'heights': tensor([[[2.4720],
          [0.8930],
          [0.3210],
          ...,
          [2.5760],
          [1.7300],
          [1.6850]]]),
 'x': tensor([[[-0.5166,  1.4814,  2.5766],
          [ 1.1278,  1.7540,  0.9303],
          [ 1.0922,  1.7178,  0.3414],
          ...,
          [-1.7778, -0.3406,  2.6815],
          [-1.7814, -0.3256,  1.8128],
          [-1.7815, -0.3236,  1.7574]]])}

In [18]:
test["x"] = get_features_by_keys(test, keys='x')

In [20]:
test['x'].shape

torch.Size([1, 3, 12766])

In [21]:
test = {k: v.to("cuda") for k, v in test.items()}

In [22]:
test["x"].shape

torch.Size([1, 3, 12766])

In [23]:
model = model.to("cuda")

In [25]:
res = model(test)
res.shape

torch.Size([1, 13, 12766])

In [ ]:
label = torch.argmax(res, dim=1)
label.shape

In [ ]:
test["pos"].shape

In [ ]:
import matplotlib.pyplot as plt

ax = plt.figure().add_subplot(projection="3d")

ax.scatter(test["pos"][0, :, 0].cpu(), test["pos"][0, :, 1].cpu(), test["pos"][0, :, 2].cpu(), c = label[0].cpu(), s=0.1)

In [ ]:
batch